In [1]:
# Requried Libraries

from urllib.request import urlopen
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np

In [2]:
# Load cik_list.xlsx data

data = pd.read_excel("D:/Work/Blackcoffer/cik_list.xlsx",engine='openpyxl')
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [3]:
data.shape

(8762, 6)

In [4]:
# Droping null values

data.isnull().sum()
data.dropna(inplace=True,axis=0)
data.shape

(152, 6)

In [5]:
# Preparing url column

data["SECFNAME"] = "https://www.sec.gov/Archives/" + data["SECFNAME"]
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...


In [6]:
# Downloading text files
# I am commenting this because i have alredy downloded that

# for i in data["SECFNAME"]:
#     r = urllib.request.urlretrieve(i, "D:/Work/Blackcoffer/Test1/{}".format(str(i).split("/")[-1]))

In [7]:
# Text extraction patterns

mda_regex = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qqd_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
            r"Market Risk.*?^\s*item\s*\d\s*"
riskfactor_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

In [8]:
# Filepath locations

stopWordsFile = 'D:/Work/Blackcoffer/StopWords_Generic.txt'
positiveWordsFile = 'D:/Work/Blackcoffer/PositiveWords.txt'
nagitiveWordsFile = 'D:/Work/Blackcoffer/NegativeWords.txt'
uncertainty_dictionaryFile = 'D:/Work/Blackcoffer/uncertainty_dictionary.txt'
constraining_dictionaryFile = 'D:/Work/Blackcoffer/constraining_dictionary.txt'

In [9]:
# Function for extracting requried text

def rawdata_extract(path, cikListFile):
    html_regex = re.compile(r'<.*?>')
    extraxted_data=[]
    
    
    cikListFile = pd.read_excel(cikListFile,engine="openpyxl")
    for index, row in cikListFile.iterrows():
        processingFile=str(row['SECFNAME']).split('/')
        inputFile = list(processingFile)[-1]
        cik=row['CIK']
        coname=row['CONAME']
        fyrmo=row['FYRMO']
        fdate = row['FDATE']
        form = row['FORM']
        secfname=row['SECFNAME']
        for fileName in os.listdir(path):
            filenameopen = os.path.join(path, fileName)
            dirFileName = filenameopen.split('\\')
            currentFile=dirFileName[1]

            if os.path.isfile(filenameopen) and currentFile == inputFile :
                resultdict = dict()
                resultdict['CIK'] = cik
                resultdict['CONAME'] = coname
                resultdict['FYRMO'] = fyrmo
                resultdict['FDATE'] = fdate
                resultdict['FORM'] = form
                resultdict['SECFNAME'] = secfname
                
                with open(filenameopen, 'r', encoding='utf-8', errors="replace") as in_file:
                    content = in_file.read()
                    content = re.sub(html_regex,'',content)
                    content = content.replace('&nbsp;','')
                    content = re.sub(r'&#\d+;', '', content)
                    matches_mda = re.findall(mda_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_mda:
                        result = max(matches_mda, key=len)
                        result = str(result).replace('\n', '')
                        resultdict['mda_extract'] = result
                    else:
                        resultdict['mda_extract'] = ""
                    match_qqd = re.findall(qqd_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if match_qqd:
                        result_qqd = max(match_qqd, key=len)
                        result_qqd = str(result_qqd).replace('\n','')
                        resultdict['qqd_extract']= result_qqd
                    else:
                        resultdict['qqd_extract'] = ""
                    match_riskfactor = re.findall(riskfactor_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if match_riskfactor:
                        result_riskfactor = max(match_riskfactor, key=len)
                        result_riskfactor = str(result_riskfactor).replace('\n', '')
                        resultdict['riskfactor_extract'] = result_riskfactor
                    else:
                        resultdict['riskfactor_extract'] = ""
                    extraxted_data.append(resultdict)

                in_file.close()

    return extraxted_data

## Section 1.1: Positive score, negative score, polarity score

#### Loading stop words dictionary for removing stop words

In [10]:
# open stopwords file

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []

#### tokenizeing module and filtering tokens using stop words list, removing punctuations

In [11]:
# Tokenizer

def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

### Positive Words

In [12]:
# Loading positive words

with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

In [13]:
# Calculating positive score

def positive_score(text):
    numPosWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in positiveWordList:
            numPosWords  += 1
    
    sumPos = numPosWords
    return sumPos

### Negative Words

In [14]:
# Loading negative words

with open(nagitiveWordsFile ,'r') as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

In [15]:
# Calculating Negative score

def negative_word(text):
    numNegWords=0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in negativeWordList:
            numNegWords -=1
    sumNeg = numNegWords 
    sumNeg = sumNeg * -1
    return sumNeg

### Polarity

In [16]:
# Calculating polarity score

def polarity_score(positiveScore, negativeScore):
    pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    return pol_score

## Section 2 -Analysis of Readability - Average Sentence Length, percentage of complex words, fog index

In [17]:
# Calculating Average sentence length 

# It will calculated using formula --- Average Sentence Length = the number of words / the number of sentences
     
def average_sentence_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

In [18]:
# Calculating percentage of complex word 

# It is calculated using Percentage of Complex words = the number of complex words / the number of words 

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

In [19]:
# calculating Fog Index 

# Fog index is calculated using -- Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

## Section 4: Complex word count

In [20]:
# Counting complex words

def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

## Section 5: Word count

In [21]:
#Counting total words

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

In [22]:
# opening uncertainty file

with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
    uncertainDict=uncertain_dict.read().lower()
uncertainDictionary = uncertainDict.split('\n')

In [23]:
# calculating uncertainty_score

def uncertainty_score(text):
    uncertainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in uncertainDictionary:
            uncertainWordnum +=1
    sumUncertainityScore = uncertainWordnum 
    
    return sumUncertainityScore

In [24]:
# opening constraining file

with open(constraining_dictionaryFile ,'r') as constraining_dict:
    constrainDict=constraining_dict.read().lower()
constrainDictionary = constrainDict.split('\n')

In [25]:
# calculating constraining score

def constraining_score(text):
    constrainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnum +=1
    sumConstrainScore = constrainWordnum 
    
    return sumConstrainScore

In [26]:
# Calculating positive word proportion

def positive_word_prop(positiveScore,wordcount):
    positive_word_proportion = 0
    if wordcount !=0:
        positive_word_proportion = positiveScore / wordcount
        
    return positive_word_proportion

In [27]:
# Calculating negative word proportion

def negative_word_prop(negativeScore,wordcount):
    negative_word_proportion = 0
    if wordcount !=0:
        negative_word_proportion = negativeScore / wordcount
        
    return negative_word_proportion

In [28]:
# Calculating uncertain word proportion

def uncertain_word_prop(uncertainScore,wordcount):
    uncertain_word_proportion = 0
    if wordcount !=0:
        uncertain_word_proportion = uncertainScore / wordcount
        
    return uncertain_word_proportion

In [29]:
# Calculating constraining word proportion

def constraining_word_prop(constrainingScore,wordcount):
    constraining_word_proportion = 0
    if wordcount !=0:
        constraining_word_proportion = constrainingScore / wordcount
        
    return constraining_word_proportion

In [30]:
# calculating Constraining words for whole report

def constrain_word_whole(mdaText,qqdmrText,rfText):
    wholeDoc = mdaText + qqdmrText + rfText
    constrainWordnumWhole =0
    rawToken = tokenizer(wholeDoc)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnumWhole +=1
    sumConstrainScoreWhole = constrainWordnumWhole 
    
    return sumConstrainScoreWhole

## Applying All Functions

In [31]:
inputDirectory = 'D:/Work/Blackcoffer/Test1'
masterFile = 'D:/Work/Blackcoffer/cik_list.xlsx'
dataList = rawdata_extract( inputDirectory , masterFile )
df = pd.DataFrame(dataList)

df['mda_positive_score'] = df.mda_extract.apply(positive_score)
df['mda_negative_score'] = df.mda_extract.apply(negative_word)
df['mda_polarity_score'] = np.vectorize(polarity_score)(df['mda_positive_score'],df['mda_negative_score'])
df['mda_average_sentence_length'] = df.mda_extract.apply(average_sentence_length)
df['mda_percentage_of_complex_words'] = df.mda_extract.apply(percentage_complex_word)
df['mda_fog_index'] = np.vectorize(fog_index)(df['mda_average_sentence_length'],df['mda_percentage_of_complex_words'])
df['mda_complex_word_count']= df.mda_extract.apply(complex_word_count)
df['mda_word_count'] = df.mda_extract.apply(total_word_count)
df['mda_uncertainty_score']=df.mda_extract.apply(uncertainty_score)
df['mda_constraining_score'] = df.mda_extract.apply(constraining_score)
df['mda_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['mda_positive_score'],df['mda_word_count'])
df['mda_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['mda_negative_score'],df['mda_word_count'])
df['mda_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['mda_uncertainty_score'],df['mda_word_count'])
df['mda_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['mda_constraining_score'],df['mda_word_count'])

df['qqdmr_positive_score'] = df.qqd_extract.apply(positive_score)
df['qqdmr_negative_score'] = df.qqd_extract.apply(negative_word)
df['qqdmr_polarity_score'] = np.vectorize(polarity_score)(df['qqdmr_positive_score'],df['qqdmr_negative_score'])
df['qqdmr_average_sentence_length'] = df.qqd_extract.apply(average_sentence_length)
df['qqdmr_percentage_of_complex_words'] = df.qqd_extract.apply(percentage_complex_word)
df['qqdmr_fog_index'] = np.vectorize(fog_index)(df['qqdmr_average_sentence_length'],df['qqdmr_percentage_of_complex_words'])
df['qqdmr_complex_word_count']= df.qqd_extract.apply(complex_word_count)
df['qqdmr_word_count'] = df.qqd_extract.apply(total_word_count)
df['qqdmr_uncertainty_score']=df.qqd_extract.apply(uncertainty_score)
df['qqdmr_constraining_score'] = df.qqd_extract.apply(constraining_score)
df['qqdmr_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['qqdmr_positive_score'],df['qqdmr_word_count'])
df['qqdmr_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['qqdmr_negative_score'],df['qqdmr_word_count'])
df['qqdmr_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['qqdmr_uncertainty_score'],df['qqdmr_word_count'])
df['qqdmr_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['qqdmr_constraining_score'],df['qqdmr_word_count'])

df['rf_positive_score'] = df.riskfactor_extract.apply(positive_score)
df['rf_negative_score'] = df.riskfactor_extract.apply(negative_word)
df['rf_polarity_score'] = np.vectorize(polarity_score)(df['rf_positive_score'],df['rf_negative_score'])
df['rf_average_sentence_length'] = df.riskfactor_extract.apply(average_sentence_length)
df['rf_percentage_of_complex_words'] = df.riskfactor_extract.apply(percentage_complex_word)
df['rf_fog_index'] = np.vectorize(fog_index)(df['rf_average_sentence_length'],df['rf_percentage_of_complex_words'])
df['rf_complex_word_count']= df.riskfactor_extract.apply(complex_word_count)
df['rf_word_count'] = df.riskfactor_extract.apply(total_word_count)
df['rf_uncertainty_score']=df.riskfactor_extract.apply(uncertainty_score)
df['rf_constraining_score'] = df.riskfactor_extract.apply(constraining_score)
df['rf_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['rf_positive_score'],df['rf_word_count'])
df['rf_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['rf_negative_score'],df['rf_word_count'])
df['rf_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['rf_uncertainty_score'],df['rf_word_count'])
df['rf_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['rf_constraining_score'],df['rf_word_count'])

df['constraining_words_whole_report'] = np.vectorize(constrain_word_whole)(df['mda_extract'],df['qqd_extract'],df['riskfactor_extract'])


In [32]:
df.shape

(152, 52)

## Final Output

In [33]:
inputTextCol = ['mda_extract','qqd_extract','riskfactor_extract']
finalOutput = df.drop(inputTextCol,1)

finalOutput.head(50)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,17,61,-0.564103,24,...,0.000000,0,0,0,0,0,0,0,0,10
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,9,46,-0.672727,30,...,0.000000,0,0,0,0,0,0,0,0,2
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0.000000,0,...,0.000000,0,0,0,0,0,0,0,0,0
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,41,119,-0.487500,23,...,0.000000,0,0,0,0,0,0,0,0,39
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0.000000,0,...,0.000000,0,0,0,0,0,0,0,0,0
5,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-25,10-Q/A,edgar/data/3662/0000950170-98-002278.txt,19,63,-0.536585,23,...,0.000000,0,0,0,0,0,0,0,0,23
6,3662.0,SUNBEAM CORP/FL/,199812.0,1998-12-22,10-Q,edgar/data/3662/0000950170-98-002401.txt,40,106,-0.452055,22,...,0.000000,0,0,0,0,0,0,0,0,34
7,3662.0,SUNBEAM CORP/FL/,199812.0,1998-12-22,10-Q,edgar/data/3662/0000950170-98-002402.txt,38,102,-0.457143,22,...,0.000000,0,0,0,0,0,0,0,0,32
8,3662.0,SUNBEAM CORP/FL/,199903.0,1999-03-31,NT 10-K,edgar/data/3662/0000950172-99-000362.txt,0,0,0.000000,0,...,0.000000,0,0,0,0,0,0,0,0,0
9,3662.0,SUNBEAM CORP/FL/,199905.0,1999-05-11,10-K,edgar/data/3662/0000950170-99-000775.txt,71,270,-0.583578,23,...,0.000000,0,0,0,0,0,0,0,0,74


## Saving CSV File

In [34]:
finalOutput.to_csv('Final Output.csv', sep=',', encoding='utf-8')